# Predicting Car Insurance Claims: A Comparative Analysis of Machine Learning Models with Final Chosen Model

As a risk department of an insurance company, our main goal is to ensure the stability of our cashflows and minimize the risk of losses. To achieve this, we are using a dataset with insurance data for cars to build a model that predicts whether a claim will be made in the first 6 months of insurance.

By predicting the likelihood of a claim, we can take proactive measures to mitigate the risk and minimize the potential losses. For example, if our model predicts a high likelihood of a claim, we can increase the premium or set a higher deductible to reduce the potential payout. On the other hand, if the model predicts a low likelihood of a claim, we can offer competitive prices to attract more customers while maintaining our profitability.

In this context, the best metric for evaluating the model would be the area under the receiver operating characteristic curve (AUC-ROC). This metric takes into account both the true positive rate (sensitivity) and the false positive rate (1-specificity) of the model, which is important for balancing the cost of false positives (unnecessary increase in premiums) and false negatives (unexpected payouts). A higher AUC-ROC score indicates a better performing model that can help us make more accurate predictions and manage the risk effectively.